In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import scipy.special as spec
import scipy.integrate as integrate
import scipy.optimize as opt
#data array

#burrell data
Experimental = np.array(
                #burrell
                [('burrell', 'C40+ optical', 99.99, 145, 1.168, 1), ('benhelm', 'C40+ optical', 99.925, 3000, 1.168, 1), ('myerson', 'C40+ optical', 99.991, 145, 1.168, 1),],

            #fidelity the average is for the two states (burrell)
            dtype=[('author', np.string_), ('ion', np.string_), ('fidelity', 'float'), ('readout time(us)', 'float'), ('coherence T1(s)', 'U8'), ('coherence T2(ms)','U8')])

print(Experimental)

[(b'', b'', 99.99 ,  145., '1.168', '1')
 (b'', b'', 99.925, 3000., '1.168', '1')
 (b'', b'', 99.991,  145., '1.168', '1')]


In [2]:
'''In general, the rates for pumping between the qubit states will not be zero. For the
calcium optical qubit the |B〉→|D〉 pumping rate remains negligible (section 4.3.3), but
the rate of |D〉→|B〉pumping due to decay from the metastable D5/2 state is appreciable
at 1/τ where τ = 1.168(7) s is the average lifetime of the shelf state [138].
The count distribution B0(n) for trials prepared in the |B〉 state remains Poisson 
distributed with mean (RB + RD)tb, but the distribution for trials initially prepared in
the |D〉 state is modified to contain two terms:'''

'In general, the rates for pumping between the qubit states will not be zero. For the\ncalcium optical qubit the |B〉→|D〉 pumping rate remains negligible (section 4.3.3), but\nthe rate of |D〉→|B〉pumping due to decay from the metastable D5/2 state is appreciable\nat 1/τ where τ = 1.168(7) s is the average lifetime of the shelf state [138].\nThe count distribution B0(n) for trials prepared in the |B〉 state remains Poisson \ndistributed with mean (RB + RD)tb, but the distribution for trials initially prepared in\nthe |D〉 state is modified to contain two terms:'

In [3]:
'''The number of counts n recorded in tb when an ion decays at a time t is Poisson distributed with mean λ(t)'''

#functions

def Bmean(RB, RD, tb):
    x = (RB+RD)*tb
    return x

def Dmean(RD, tb):
    x = RD*tb
    return x

In [4]:
#Optimal Time vs Collection Efficiency

#Find error at a specific bin time and n count

Rb = 55800
Rd = 442
#max bin time
tb = .00042
nc = 50
#optimal count nc
tau = 1.168 #s for Ca40


#integrate D over n's @tb
def ed(tb): 
    return integrate.quad(D_integrand, nc, nc*10, args=(Rd,Rb,tb,tau))[0]

def D_integrand(n,Rd,Rb,tb,tau):
    return ((np.e**(-1*tb/tau)*((Dmean)**n)*(np.exp(Dmean)) / spec.factorial(n)) + (integrate.quad(X_integrand, Rd*tb, (Rb+Rd)*tb, args=(n,Rb,Rd,tau,tb)))[0])

#integrate X over lambda's
def X_integrand(l,n,Rb,Rd,tau,tb):
    return (1/(Rb*tau))*np.exp((l-((Rd+Rb)*tb))/(Rb*tau))*np.power(l,n)*np.exp(-1*l)/spec.factorial(n)


#0 to nc w/ B0
def eb(tb):
    return (integrate.quad(B_integrand, 0, nc, args=(Rb, Rd, tb))[0])

def B_integrand(Rb, Rd, tb, n):
    return (np.power((Rb+Rd)*tb,n)) * (np.exp(-1*((Rb+Rd)*tb))) / spec.factorial(n)

tb = np.arange(start=.00001,stop=tb*2,step=.00001)
#print(tb)

darkE = np.empty(0)
brightE = np.empty(0)

for i in tb:
    darkE = np.append(darkE, ed(i))
for i in tb:
    brightE = np.append(brightE, eb(i))

print(darkE)

print(brightE)

fig = plt.figure(figsize=(10, 5))
ax1 = fig.add_subplot(1, 1, 1)
ax1.plot(tb, darkE, lw=0.75, c="r", label = "D0")
ax1.set_ylabel("Error")
ax1.set_xlabel("Bin Time (s)")
ax1.plot(tb, brightE, lw=0.75, c="b", label = "B0")
ax1.legend(loc = 'upper center')

plt.title("Dark and Bright Errors Finite Lifetime ")


TypeError: unsupported operand type(s) for ** or pow(): 'function' and 'float'